In [1]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

--2023-07-22 05:01:30--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56286 (55K) [text/plain]
Saving to: ‘robert_frost.txt’

robert_frost.txt    100%[===================>]  54.97K  --.-KB/s    in 0.01s   

2023-07-22 05:01:30 (5.07 MB/s) - ‘robert_frost.txt’ saved [56286/56286]



In [2]:
frost = []
alphabets = [' ','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

with open('robert_frost.txt') as file:
  for line in file.readlines():
    line = line.replace('\n','')
    line = line.lower()
    for ch in line:
      if ch not in alphabets:
        line = line.replace(ch,'')
    if len(line)!=0:
      frost.append(line.split())

In [3]:
import numpy as np
np.random.seed(1234)

In [4]:
def add2dic(d,k,v):
  if k not in d:
    d[k] = []
  d[k].append(v)

In [5]:
pi = {}
A1 = {}
A2 = {}

for line in frost:
  T = len(line)

  for i in range(T):
    if i==0 :
      pi[line[0]] = pi.get(line[0],0.)+1
    else:

      if i==T-1:
        add2dic(A2, (line[i-1],line[i]), 'END')

      if i==1:
        add2dic(A1,line[i-1],line[i])
      else:
        add2dic(A2, (line[i-2],line[i-1]), line[i])

for keys in pi:
  pi[keys]=pi[keys]/len(frost)

In [6]:
# convert something like [cat, cat, cat, dog, dog, dog, dog, mouse, ...]
# into {cat: 0.5, dog: 0.4, mouse: 0.1}

def list2dic(tokens):
  d = {}
  n = len(tokens)
  for token in tokens:
    d[token] = d.get(token, 0.) + 1
  for key in d:
    d[key]/=n
  return d

In [7]:
for key,value in A1.items():
  A1[key] = list2dic(value)

In [8]:
for key, value in A2.items():
  A2[key] = list2dic(value)

In [9]:
# if {a:0.2 b:0.5 c:0.3}
# then distrivution is like
# 0<=p<=0.2 a
# 0.2<=p<=0.7 b
# 0.7<=p<=1 c
# so we are sampling the word based on their weights

def sample_word(d):
  p0 = np.random.random()
  cummulative = 0
  for t,p in d.items():
    cummulative += p
    if p0 < cummulative:
      return t
  assert(False)

In [10]:
def generate():
  for i in range(4):
    sentence = ""
    #1st word is independent
    w0 = sample_word(pi)
    sentence+=w0
    #2nd word is dependent only on 1st
    w1=sample_word(A1[w0])
    sentence+=" "+w1
    # the rest of the words are dependent on the 2 words before them
    while True:
      w2 = sample_word(A2[(w0,w1)])
      if w2 == 'END':
        break
      sentence+=" "+w2
      w0 = w1
      w1 = w2
    print(sentence)

In [11]:
generate()

i went to bed alone and left me
might just as empty
but it isnt as if and thats not all the money goes so fast
you couldnt call it living for it aint
